In [1]:
def sph2cart(r,theta,phi):
    rcos_theta = r * np.cos(theta)
    x = rcos_theta * np.cos(phi)
    y = rcos_theta * np.sin(phi)
    z = r * np.sin(theta)
    return x, y, z

def cart2sph(x,y,z):
    XsqPlusYsq = x**2 + y**2
    r = math.sqrt(XsqPlusYsq + z**2)               # r
    elev = math.atan2(z,math.sqrt(XsqPlusYsq))     # theta
    az = math.atan2(y,x)                           # phi
    return r, elev, az

In [4]:
import os

In [8]:
os.listdir("static")

['favicon.ico', 'test.bmp', 'bottle.depth.tiff', 'bottle.jpg', 'index.css']

In [2]:
from PIL import Image
import numpy as np
import math
import open3d as o3d
from open3d import JVisualizer
from multiprocessing.pool import Pool
import pymesh
import os

In [3]:
img = Image.open("static/bottle.depth.tiff")
img_bak = img
img.show()

In [9]:
# Lower depth map resolution
nH = img.height//4
nW = img.width//4
img = img.resize((nW,nH),Image.ANTIALIAS)

In [10]:
img.show()

In [11]:
I = np.asarray(img)

In [13]:
I.shape
height,width = I.shape
#height,width,channels = I.shape
center_x = width//2
center_y = height//2

In [14]:
I[0].shape

(75,)

In [23]:
pool = Pool(3)
def ptGen(tup):
    x,y,z,r = tup
    max_r , theta , phi = cart2sph(x,y,z)
    r_normalized = (r/256) * max_r
    x,y,z = sph2cart(r_normalized,theta,phi)
    return [x,y,z]
    
queries = []
for i in range(height):
    for j in range(width):
        x = j - center_x
        y = center_y - i
        z = 200
        r = 256 - I[i][j]
        #r = I[i][j][0]
        
        queries.append((x,y,z,r))  
        
pt_cloud = pool.map( ptGen , queries)
pool.close()

In [24]:
pt_cloud[:10]

[[-1.01171875, 1.3671875000000002, 5.468749999999999],
 [-0.9843749999999999, 1.3671874999999998, 5.46875],
 [-0.9570312500000002, 1.3671874999999998, 5.46875],
 [-0.9296874999999994, 1.3671874999999998, 5.46875],
 [-0.9023437499999999, 1.3671874999999998, 5.46875],
 [-0.8750000000000003, 1.3671875, 5.46875],
 [-0.8476562500000004, 1.3671875000000002, 5.468749999999999],
 [-0.8203124999999996, 1.3671874999999998, 5.468750000000001],
 [-1.3593750000000004, 2.3437500000000004, 9.375],
 [-1.6406250000000002, 2.9296874999999996, 11.71875]]

In [25]:
pt_cloud[:10]

[[-1.01171875, 1.3671875000000002, 5.468749999999999],
 [-0.9843749999999999, 1.3671874999999998, 5.46875],
 [-0.9570312500000002, 1.3671874999999998, 5.46875],
 [-0.9296874999999994, 1.3671874999999998, 5.46875],
 [-0.9023437499999999, 1.3671874999999998, 5.46875],
 [-0.8750000000000003, 1.3671875, 5.46875],
 [-0.8476562500000004, 1.3671875000000002, 5.468749999999999],
 [-0.8203124999999996, 1.3671874999999998, 5.468750000000001],
 [-1.3593750000000004, 2.3437500000000004, 9.375],
 [-1.6406250000000002, 2.9296874999999996, 11.71875]]

In [26]:
pt_arr= np.array(pt_cloud)
pt_arr.shape

(7500, 3)

In [27]:
o3d_cloud = o3d.geometry.PointCloud()

In [28]:
o3d_cloud.points = o3d.utility.Vector3dVector(pt_arr)

In [29]:
o3d.visualization.draw_geometries_with_editing([o3d_cloud])

In [22]:
len(pt_cloud)

7500

In [18]:
texture_verts = []
mesh = []

for i in range(height):
    for j in range(width):
        tex = [
            j / width,
            1 - i / height
        ]
        texture_verts.append(tex)

for i in range(height - 1 ):
    for j in range(width - 1):
        triangle1 = [
            i*width + j  ,
            i*width + (j + 1) ,
            (i+1)* width + j
        ]
        triangle1.reverse()
        
        triangle2 = [
            i*width + (j + 1) ,
            (i+1) * width + (j+1),
            (i+1)* width + j
        ]
        triangle2.reverse()
        
        mesh.append(triangle1)
        mesh.append(triangle2)

        

In [19]:
mesh_arr = np.array(mesh)

In [20]:
mesh_arr.shape

(40874, 3)

In [21]:
o3d_mesh = o3d.geometry.TriangleMesh()

In [22]:
o3d_mesh.vertices= o3d.utility.Vector3dVector(pt_arr)

In [23]:
o3d_mesh.triangles = o3d.utility.Vector3iVector(mesh_arr)

In [24]:
o3d.visualization.draw_geometries_with_editing([o3d_mesh])

In [25]:
o3d_lines = o3d.geometry.LineSet.create_from_triangle_mesh(o3d_mesh)

In [26]:
o3d.visualization.draw_geometries_with_editing([o3d_lines])

In [115]:
temp = o3d_mesh.filter_smooth_taubin()

In [116]:
o3d.visualization.draw_geometries_with_editing([temp])

In [358]:
def writeObj(verts,faces,fName="test.obj"):
    f = open(fName,"w+")
    for vert in verts:
        x,y,z = vert
        f.write("v {} {} {}\n".format(x/1000,y/1000,z/1000))
        
    for t_vert in texture_verts:
        u,v = t_vert
        f.write("vt {} {}\n".format(u,v))
    
    for face in faces:
        a,b,c = face
        f.write("f {}/{} {}/{} {}/{}\n".format(a+1,a+1,b+1,b+1,c+1,c+1))
    f.close()
        

In [359]:
writeObj(pt_arr,mesh_arr)

In [128]:
os.getcwd()

'/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/Resources/Python.app/Contents/Resources'

In [129]:
mesh_arr.shape

(4141202, 3)

In [130]:
pt_arr.shape

(2073600, 3)

In [364]:
t = Image.open("/Users/rjpj/downloads/DM/Paper lantern.png")


In [365]:
t.show()

In [367]:
t_r = t.resize((1920,1920),Image.ANTIALIAS)

In [368]:
t_r.save("/Users/rjpj/downloads/DM/Paper lantern square.png")

In [369]:
t_r.show()